# Main Confusions While Working on AWS Ec2 Instance

In [1]:
import boto3
import json

# Create EC2 client
ec2 = boto3.client('ec2')

# Call describe_instances
response = ec2.describe_instances()      # This is a dictionary with main 2 keys : Reservations and ResponseMetadata

# Optional: Pretty-print full output like aws cli
print(json.dumps(response, indent=4,default = str))


{
    "Reservations": [
        {
            "ReservationId": "r-0887bec7c2e48b1e8",
            "OwnerId": "120569645542",
            "Groups": [],
            "Instances": [
                {
                    "Architecture": "x86_64",
                    "BlockDeviceMappings": [
                        {
                            "DeviceName": "/dev/sda1",
                            "Ebs": {
                                "AttachTime": "2025-07-10 14:25:48+00:00",
                                "DeleteOnTermination": true,
                                "Status": "attached",
                                "VolumeId": "vol-0b8d295d79a7ffb83"
                            }
                        }
                    ],
                    "ClientToken": "b5df8917-5e2a-42ed-ad9f-0d888f67fd4b",
                    "EbsOptimized": false,
                    "EnaSupport": true,
                    "Hypervisor": "xen",
                    "NetworkInterfaces": [
                

In [4]:
# There are 2 Main Keys in response dictionary
for key in response:
    print(key)

Reservations
ResponseMetadata


In [5]:
# Type of Value of response['Reservations'] is list of dictionary. Because response itself is a dictionary
print(f"Type of Value of response['Reservations'] vlaues is: ", type(response['Reservations'])) # This is a dictionary
print(f"The len of Reservation Values in list: ", len (response['Reservations'])) # The len shows the number of instances running
# if and only if they are created individually
# But If Multiple Instances are Created at once. The length of list will Still be 1. But have Multiple Instance Id 1 for each ec2
print(f"Type of Value of response['ResponseMetadata'] is: ", type(response['ResponseMetadata']))

Type of Value of response['Reservations'] vlaues is:  <class 'list'>
The len of Reservation Values in list:  2
Type of Value of response['ResponseMetadata'] is:  <class 'dict'>


In [7]:
# There are only 4 Keys in response['Reservations']
for key in response['Reservations'][0]: # Because the returned value is a list. that's why we picked the 1 element to get keys
    print(key) 

ReservationId
OwnerId
Groups
Instances


In [6]:
ReservationValue_list = response['Reservations']    # These are values in Reservatins key only and is python list

### Most Common Monitoring Parameter from Ec2 Instance

In [67]:
# These are all the keys and values in Instances key of response['Reservations']
for reser in response['Reservations']:
    for key in reser['Instances'][0]:
        print(key,": ", reser['Instances'][0][key])
    break

Architecture :  x86_64
BlockDeviceMappings :  [{'DeviceName': '/dev/sda1', 'Ebs': {'AttachTime': datetime.datetime(2025, 7, 10, 14, 25, 48, tzinfo=tzutc()), 'DeleteOnTermination': True, 'Status': 'attached', 'VolumeId': 'vol-0b8d295d79a7ffb83'}}]
ClientToken :  b5df8917-5e2a-42ed-ad9f-0d888f67fd4b
EbsOptimized :  False
EnaSupport :  True
Hypervisor :  xen
NetworkInterfaces :  [{'Association': {'IpOwnerId': 'amazon', 'PublicDnsName': 'ec2-54-211-157-154.compute-1.amazonaws.com', 'PublicIp': '54.211.157.154'}, 'Attachment': {'AttachTime': datetime.datetime(2025, 7, 10, 14, 25, 47, tzinfo=tzutc()), 'AttachmentId': 'eni-attach-06c381de916780bb7', 'DeleteOnTermination': True, 'DeviceIndex': 0, 'Status': 'attached', 'NetworkCardIndex': 0}, 'Description': '', 'Groups': [{'GroupId': 'sg-08ab6d928e1529143', 'GroupName': 'launch-wizard-3'}], 'Ipv6Addresses': [], 'MacAddress': '12:56:96:d0:42:cf', 'NetworkInterfaceId': 'eni-0e0e8964af590bce7', 'OwnerId': '120569645542', 'PrivateDnsName': 'ip-172-

In [68]:
# dict.get(key) is used when we are not sure whether the key will exists or not. Good Practice to be safe
# dict.get(key,[]) # In the above case if no key found returns None. while in this syntax returns empty list. Good Practice.
total_ec2 = 0
for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        total_ec2 += 1
        #General Most Important to Monitor Regularly
        print(f"ID: {instance['InstanceId']}")
        print(f"Name: {[tag['Value'] for tag in instance.get('Tags', []) if tag['Key'] == 'Name']}") # Tags is a dictionary
        print(f"Type: {instance['InstanceType']}")
        print(f"State: {instance['State']['Name']}")
        print(f"Private IP: {instance.get('PrivateIpAddress')}")
        print(f"Public IP: {instance.get('PublicIpAddress', 'N/A')}")
        print(f"AZ: {instance['Placement']['AvailabilityZone']}") # The Value itself is a Dictioary 
        print(f"Key Pair: {instance.get('KeyName')}")
        print(f"Launch Time: {instance['LaunchTime']}")
        print()
        eni = instance['NetworkInterfaces'][0]  # Assuming 1st ENI

        #These are the Most Common From NetworkInterface key in Instances Key 
        print("ENI ID:", eni['NetworkInterfaceId'])
        print("Private IP:", eni['PrivateIpAddress'])
        print("Public IP:", eni.get('Association', {}).get('PublicIp', 'None'))
        print("Public DNS:", eni.get('Association', {}).get('PublicDnsName', 'None'))
        print("Subnet ID:", eni['SubnetId'])
        print("VPC ID:", eni['VpcId'])
        print("MAC Address:", eni['MacAddress'])
        print("Security Group(s):", [g['GroupName'] for g in eni['Groups']])
        print("Delete on Termination:", eni['Attachment']['DeleteOnTermination'])
        print("Status:", eni['Status'])
        print("Source/Dest Check:", eni['SourceDestCheck'])
                
        print("-" * 40)
    print()
    print(f"The Total Ec2 Instacces Created are: ", total_ec2)

ID: i-0d62ab2e0367f1cac
Name: ['MyInstance']
Type: t2.micro
State: running
Private IP: 172.31.89.83
Public IP: 54.211.157.154
AZ: us-east-1b
Key Pair: keyPair-MyInstance
Launch Time: 2025-07-10 14:25:47+00:00

ENI ID: eni-0e0e8964af590bce7
Private IP: 172.31.89.83
Public IP: 54.211.157.154
Public DNS: ec2-54-211-157-154.compute-1.amazonaws.com
Subnet ID: subnet-037d4834473ad1480
VPC ID: vpc-0d3865d7d40a5d547
MAC Address: 12:56:96:d0:42:cf
Security Group(s): ['launch-wizard-3']
Delete on Termination: True
Status: in-use
Source/Dest Check: True
----------------------------------------

The Total Ec2 Instacces Created are:  1
ID: i-03da77c39f993ed10
Name: ['Instance2']
Type: t2.micro
State: running
Private IP: 172.31.86.90
Public IP: 52.207.233.177
AZ: us-east-1b
Key Pair: Instance2-KeyPair
Launch Time: 2025-07-10 20:31:09+00:00

ENI ID: eni-0f5341923cfba7299
Private IP: 172.31.86.90
Public IP: 52.207.233.177
Public DNS: ec2-52-207-233-177.compute-1.amazonaws.com
Subnet ID: subnet-037d483

In [8]:
pip install pandoc


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/128.0 kB ? eta -:--:--
     ---------                               30.7/128.0 kB 1.3 MB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------                           41.0/128.0 kB 495.5 kB/s eta 0:00:01
     -----------         


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
